In [ ]:
!pip install pydub
import numpy as np
import zipfile
import glob as gb
import librosa as lb
import os
import soundfile as sf
from pydub import AudioSegment

In [ ]:
!unzip -uq "/content/drive/MyDrive/rawdess.zip" -d "/content/speech/"


In [ ]:
!unzip -uq "/content/drive/MyDrive/urbansound.zip" -d "/content/speech/"

In [ ]:
audio_dir = '/content/speech/urbansound_8k'

In [ ]:
noise_dir = '/content/speech/ravdess_rewritten_8k'

In [ ]:
audio_files = gb.glob(os.path.join(audio_dir, '*.wav'))

In [ ]:
noise_files = gb.glob(os.path.join(noise_dir, '*.wav'))

In [ ]:
def newsound(speech_file, noise_file,output_path, sr=22050):
             speech, _ = lb.load(speech_file)
             noise, _ = lb.load(noise_file)
             speech = AudioSegment.from_wav(speech_file)
             noise = AudioSegment.from_wav(noise_file)

             if len(noise) > len(speech):
              noise = noise[:len(speech)]
             else:
              noise = noise * (len(speech) // len(noise) + 1)
              noise = noise[:len(speech)]

             speech = speech.apply_gain(-20)

             min_length = min(len(speech),len(noise))

             speech = speech[:min_length]
             noise = noise[:min_length]
             mixed = speech.overlay(noise)
             mixed.export(output_path, format="wav")





In [ ]:
def main(audio_dir, noise_dir):
    output_dir = 'mixed_sounds'
    os.makedirs(output_dir, exist_ok=True)




    for audio_file, noise_file in zip(audio_files, noise_files):
        output_path = os.path.join(output_dir, os.path.basename(audio_file))
        newsound(audio_file, noise_file, output_path)

    with zipfile.ZipFile("/content/drive/MyDrive/mixed.zip", 'w') as zipf:
        for root, _, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, output_dir))

In [ ]:
main(audio_files,noise_files)